In [1]:
import os

import numpy as np
import scipy.ndimage
from tqdm import tqdm

import skimage.io
import skimage.color
import skimage.exposure
from skimage.measure import block_reduce


import cv2

# Operations to prepare the DAVIS Dataset

Using the DAVIS 2019 unsupervised test-dev dataset from https://davischallenge.org/davis2017/code.html.

The Notebook uses the given video-frames to:
1. create grayscale images
2. resizes the images to a given size
3. compute optical flow
4. (create playable videos from the images)



TODO:

  - image resolution:
    - Davis 2019: 854x480
    - We probably want to rescale to smaller images, e.g.: 256×144, 320×180, 400×222, 512×288, 568×320, 640×360 
    
    (https://en.wikipedia.org/wiki/Low-definition_television)
  - optical flow:
    - We might want to use a different optical flow method, e.g. DeepFlow
    - We should think how to best store the optical flow images (maybe 2-channel images)
    - want to convey pixel-2-pixel correspondence - 
      need store flow such that reconstruction of vectors is possible
    - IDEA:
      - set reasonable maximum flow value (e.g. 100 pixels - 256bit img = 256px?)
      - normalize flow values to [0,1] (divide by max)
      - store as:
        - 2-channel (x,y) 
        - 3-channel (r,g,b) image 
      - reconstruct flow vector by multiplying with max
      

In [2]:
# DAVIS Dataset path specification

dataset_path = '~/Documents/Colorization/Datasets/'
dataset_name = 'DAVIS'
dataset_path = os.path.expanduser(dataset_path)
dataset_path = os.path.join(dataset_path, dataset_name)

In [3]:
# specify output image size

resize = True
source_res = "480p"
target_res = "176p"

image = skimage.io.imread(os.path.join(os.path.join(os.path.join(dataset_path, 'JPEGImages', source_res), "butterfly"), "00001.jpg"))
image_size = (image.shape[0], image.shape[1])
target_size = (176, 320)

In [4]:
def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

## 1. Converting images to grayscale

In [7]:
input_path = os.path.join(dataset_path, 'JPEGImages', source_res)
output_path = os.path.join(dataset_path, 'JPEGImages', target_res+'_gray')

make_dir(output_path)
print('Input path: {}'.format(input_path))
print('Output path: {}'.format(output_path))

subdirs = os.listdir(input_path)

Input path: /home/jansp/Documents/Colorization/Datasets/DAVIS/JPEGImages/480p
Output path: /home/jansp/Documents/Colorization/Datasets/DAVIS/JPEGImages/176p


In [8]:
# Load images of each subdirectory and convert them to grayscale

grey = True     # Disable to get resized RGB images

for subdir in tqdm(subdirs):
    subdir_path = os.path.join(input_path, subdir)
    if not os.path.exists(os.path.join(output_path, subdir)):
        os.makedirs(os.path.join(output_path, subdir))
    filenames = os.listdir(subdir_path)
    for filename in filenames:
        image_path = os.path.join(subdir_path, filename)
        image = skimage.io.imread(image_path)
        if grey:
            image = skimage.color.rgb2gray(image)
            image = skimage.exposure.rescale_intensity(image, out_range=(0, 255))
            image = image.astype(np.uint8)
        if resize:
            image = skimage.transform.resize(image, target_size, preserve_range=True)
        image = image.astype(np.uint8)
        skimage.io.imsave(os.path.join(output_path, subdir, filename), image)

100%|██████████| 30/30 [00:45<00:00,  1.51s/it]


## 2. Computing optical flows between consecutive images

In [5]:
# using original resolution images and donwsampling the calculated optical flow to target size

input_path = os.path.join(dataset_path, 'JPEGImages', source_res+'_gray')

print('Input path: {}'.format(input_path))

subdirs = os.listdir(input_path)

Input path: /home/jansp/Documents/Colorization/Datasets/DAVIS/JPEGImages/480p_gray


In [6]:
#for some reason the built-in cv2 function gives false results (inf values)

def cartToPol(x, y):  #convert cartesian to polar coordinates (for optical flow)
    ang = np.arctan2(y, x)
    mag = np.hypot(x, y)
    return mag, ang

In [7]:
# Optical flow implementation:
# Compute optical flow between consecutive frames
# code taken from: https://learnopencv2.com/optical-flow-in-opencv/

def dense_optical_flow(method, input_image_path, output_image_path, params=[], to_gray=False, save_array=False, array_path=None):
    # Read the images
    image_names = os.listdir(input_image_path)
    image_names.sort()
    old_frame = cv2.imread(os.path.join(image_path, image_names[0]))

    # create HSV & make Saturation a constant
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255

    # Convert to grayscale
    if to_gray:
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    #print('Number of frames: {}'.format(len(image_names)))
    
    for i in range(1, len(image_names)):
        # Read the next frame
        new_frame = cv2.imread(os.path.join(image_path, image_names[i]))

        # Convert to grayscale
        if to_gray:
            new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)

        # Calculate dense optical flow by Farneback method
        flow = method(old_frame, new_frame, None, *params)

        if save_array:
            flow_ds = skimage.transform.resize(flow, target_size, preserve_range=True)
            np.save(os.path.join(array_path, image_names[i]), flow_ds)
        else:
            # Encoding: convert the algorithm's output into Polar coordinates
            mag, ang = cartToPol(flow[..., 0], flow[..., 1])
            # Use Hue and Value to encode the Optical Flow
            hsv[..., 0] = (ang+np.pi) * 180 / ( 2 * np.pi )
            hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
            # Convert HSV to RGB (BGR) color representation
            bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

            # Save output image
            if resize:
                bgr = skimage.transform.resize(bgr, target_size, preserve_range=True)
            cv2.imwrite(os.path.join(output_image_path, image_names[i]), bgr)

        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break

        # Update previous frame
        old_frame = new_frame

In [8]:
# Specifying the optical flow algorithm and applying it to the dataset

# possible algorithms:  "farneback", "lucaskanade", "lucaskanade_dense", "rlof"
label = "farneback"
method = cv2.calcOpticalFlowFarneback
# params: pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags
params = [0.5, 3, 15, 3, 5, 1.2, 0]  # Farneback's algorithm parameters

output_path = os.path.join(dataset_path, 'JPEGImages', target_res+'_'+label)
array_path = os.path.join(dataset_path, 'nd_arrays', target_res+'_'+label)
make_dir(output_path)
make_dir(array_path)
print('Output path: {}'.format(output_path))
print('Array path: {}'.format(array_path))


# algorithm = "lucaskanade_dense"
# method = cv2.optflow.calcOpticalFlowSparseToDense

# Computing optical flows for each foler in the dataset:
print('Calculating dense optical flow using {} method...'.format(method.__name__))
for subdir in tqdm(subdirs):
    image_path = os.path.join(input_path, subdir)
    output_image_path = os.path.join(output_path, subdir)
    output_array_path = os.path.join(array_path, subdir)
    make_dir(output_image_path)
    make_dir(output_array_path)
    dense_optical_flow(method, image_path, output_image_path, params=params, to_gray=True, save_array=True, array_path=output_array_path)

Output path: /home/jansp/Documents/Colorization/Datasets/DAVIS/JPEGImages/176p_farneback
Array path: /home/jansp/Documents/Colorization/Datasets/DAVIS/nd_arrays/176p_farneback
Calculating dense optical flow using calcOpticalFlowFarneback method...


100%|██████████| 30/30 [02:41<00:00,  5.39s/it]


In [12]:
label = "deepflow"
deepflow = cv2.optflow.createOptFlow_DeepFlow()
output_path = os.path.join(dataset_path, 'JPEGImages', target_res+'_'+label)
array_path = os.path.join(dataset_path, 'nd_arrays', target_res+'_'+label)
make_dir(output_path)
make_dir(array_path)


# Computing optical flows for each foler in the dataset:
print('Calculating dense optical flow using {} method...'.format(deepflow.__class__.__name__))

for subdir in tqdm(subdirs):
    image_path = os.path.join(input_path, subdir)
    output_image_path = os.path.join(output_path, subdir)
    output_array_path = os.path.join(array_path, subdir)
    make_dir(output_image_path)
    make_dir(output_array_path)
    dense_optical_flow(deepflow.calc, image_path, output_image_path, params=[], to_gray=True, save_array=True, array_path=output_array_path)


Calculating dense optical flow using DenseOpticalFlow method...


100%|██████████| 30/30 [16:56<00:00, 33.90s/it]


## 3. Converting images to video

In [4]:
image_path = os.path.join(dataset_path, 'JPEGImages', '480p')
video_path = os.path.join(dataset_path, 'MP4Videos', '480p')

make_dir(video_path)

print('Image path: {}'.format(image_path))
print('Video path: {}'.format(video_path))

subdirs = os.listdir(image_path)
print('Number of subdirectories: {}'.format(len(subdirs)))

Image path: /home/jansp/Documents/Colorization/Datasets/DAVIS/JPEGImages/480p
Video path: /home/jansp/Documents/Colorization/Datasets/DAVIS/MP4Videos/480p
Number of subdirectories: 30


In [5]:
fps = 24

for subdir in tqdm(subdirs):
    image_folder = os.path.join(image_path, subdir)
    video_name = os.path.join(video_path, subdir + '.avi')
    image_names = os.listdir(image_folder)
    image_names.sort()
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, fps, (width,height))

    for image in image_names:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

  0%|          | 0/30 [00:00<?, ?it/s]


NameError: name 'images' is not defined